# **Data Preparation**

# Objectives

* Clean and remove non images.
* Prepare the dataset for further processes.
* Find average image shape for original images.
* Resize the images from the datasets.
* Split the datasets into Train, Validation and Test sets.

# Outputs

To Generate:
'inputs/leaves_dataset/cherry-leaves/resized_images'
'inputs/leaves_dataset/cherry-leaves/processed_images'

# Additional Comments

* No Additional Comments

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/PP5-Cherry-Leaves/jupyter_notebooks'

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/PP5-Cherry-Leaves'

In [4]:
import os
import shutil
import random
import joblib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
sns.set_style("white")

# Image Removal

Checks for Corrupt Images and other image types that are not '.png', '.jpg' and '.jpeg' Image Types. This helps to prevent duplicates and conflicts with different image versions (i.e original images and resized images) ready for processing.

In [7]:
import os
import tensorflow as tf

def remove_invalid_and_original_images(my_data_dir, resized_suffix="resized"):
    image_extensions = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)

    for folder in folders:
        folder_path = os.path.join(my_data_dir, folder)
        
        if not os.path.isdir(folder_path):  
            continue  # Skip if not a folder

        i, j = 0, 0  # Counters

        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            if os.path.isfile(file_path):
                if not file_name.lower().endswith(image_extensions):  
                    os.remove(file_path)  # Remove non-image files
                    i += 1
                elif resized_suffix not in file_name:  # Remove original-sized images
                    os.remove(file_path)
                    i += 1
                else:
                    try:
                        # Try opening with TensorFlow to verify it's an actual image
                        img = tf.io.read_file(file_path)
                        img = tf.io.decode_image(img, channels=3)
                        j += 1  # Valid resized image
                    except Exception as e:
                        print(f"Removing corrupted image: {file_path} - Error: {str(e)}")
                        os.remove(file_path)
                        i += 1

        print(f"Folder: {folder} - Valid Resized Images: {j}, Removed: {i}")

# Example usage:
data_dir = '/workspaces/PP5-Cherry-Leaves/inputs/leaves_dataset/cherry-leaves'
remove_invalid_and_original_images(data_dir)

Folder: powdery_mildew - Valid Resized Images: 0, Removed: 2104
Folder: healthy - Valid Resized Images: 0, Removed: 2104
Folder: resized_images - Valid Resized Images: 0, Removed: 0


Image Resizing using ImageDataGenerator prints batch size, image shape and RGB presence for visual clarification.

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator instance
datagen = ImageDataGenerator(rescale=1./255)  # Normalization (optional)

# Load images from a directory and resize them to 100x100
data_generator = datagen.flow_from_directory(
    '/workspaces/PP5-Cherry-Leaves/inputs/leaves_dataset/cherry-leaves/resized_images',  # Replace with your image directory
    target_size=(100, 100),  # Resize to 100x100
    batch_size=10,
    class_mode='binary'  # Change based on your task (e.g., 'binary' for 2 classes)
)

# Fetch a batch of images and labels
images, labels = next(data_generator)

# Check the shape
print(images.shape)  # Should be (batch_size, 100, 100, 3) for RGB images


Found 4208 images belonging to 2 classes.
(10, 100, 100, 3)


Saving images to 'processed_images' folder.

In [12]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
import numpy as np

# Define source and destination directories
source_dir = '/workspaces/PP5-Cherry-Leaves/inputs/leaves_dataset/cherry-leaves/resized_images'  # Use resized images as source
save_dir = '/workspaces/PP5-Cherry-Leaves/inputs/leaves_dataset/cherry-leaves/processed_images'  # Save final processed images

# Create ImageDataGenerator for further processing (if needed)
datagen = ImageDataGenerator(rescale=1./255)  # Normalize the images

# Create the save directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Set up the generator with target size and batch size
data_generator = datagen.flow_from_directory(
    source_dir,
    target_size=(100, 100),  # Resize to 100x100 (if necessary again)
    batch_size=32,
    class_mode='binary',  # Adjust if needed
    shuffle=False
)

# Ensure subdirectories exist for each class
for class_name in data_generator.class_indices:
    class_dir = os.path.join(save_dir, class_name)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

# Iterate over the batches of images and save them
batch_index = 0
for batch in data_generator:
    images, labels = batch
    for i, img in enumerate(images):
        # Convert the image array to a PIL image
        pil_img = array_to_img(img)
        
        # Determine the class name based on the label
        label = labels[i] 
        class_name = list(data_generator.class_indices.keys())[int(label)]
        
        # Create a filename
        img_filename = f"processed_image_{batch_index * data_generator.batch_size + i}.png"
        
        # Save the image in the appropriate class subdirectory
        pil_img.save(os.path.join(save_dir, class_name, img_filename))
    
    batch_index += 1

    # Stop after processing all images
    if batch_index * data_generator.batch_size >= data_generator.samples:
        break

print(f"Processed images saved to {save_dir}")

Found 4208 images belonging to 2 classes.


Processed images saved to /workspaces/PP5-Cherry-Leaves/inputs/leaves_dataset/cherry-leaves/processed_images


# Split the data into train, validation and test sets

Splitting the 'processed_image' data into train, validation and test subsets to a ration of 0.7, 0.15, 0.15 summing up to 1.0.

In [13]:
import random

def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):
    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # Get class labels
    labels = os.listdir(my_data_dir)
    if 'test' in labels:
        pass
    else:
        # Create train, validation, and test folders with class labels as subfolders
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=os.path.join(my_data_dir, folder, label), exist_ok=True)

        for label in labels:
            files = os.listdir(os.path.join(my_data_dir, label))
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # Move a file to the train set
                    shutil.move(os.path.join(my_data_dir, label, file_name),
                                os.path.join(my_data_dir, 'train', label, file_name))

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # Move a file to the validation set
                    shutil.move(os.path.join(my_data_dir, label, file_name),
                                os.path.join(my_data_dir, 'validation', label, file_name))

                else:
                    # Move a file to the test set
                    shutil.move(os.path.join(my_data_dir, label, file_name),
                                os.path.join(my_data_dir, 'test', label, file_name))

                count += 1

            # Remove empty class directories after splitting
            os.rmdir(os.path.join(my_data_dir, label))

# Split the resized images into train, validation, and test sets
split_train_validation_test_images(save_dir, train_set_ratio=0.7, validation_set_ratio=0.15, test_set_ratio=0.15)

print(f"Data split into train, validation, and test sets.")

Data split into train, validation, and test sets.


Comma separated values (CSV) files generated to store tabular data using plain text.

In [14]:
# Code from linx02 - genderpredictor project to create and save dataframes.

import pandas as pd

def create_dataframe(data_dir):
    data = {'file': [], 'label': []}
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        for file in os.listdir(label_dir):
            file_path = os.path.join(label_dir, file)
            data['file'].append(file_path)
            data['label'].append(label)
    return pd.DataFrame(data)

data_dir = "/workspaces/PP5-Cherry-Leaves/inputs/leaves_dataset/cherry-leaves/processed_images"

# Create DataFrames for train, validation, and test datasets
train_df = create_dataframe(os.path.join(data_dir, 'train'))
validation_df = create_dataframe(os.path.join(data_dir, 'validation'))
test_df = create_dataframe(os.path.join(data_dir, 'test'))

In [15]:
output_dir = "/workspaces/PP5-Cherry-Leaves/outputs/"

dataframes = {'train': train_df, 'validation': validation_df, 'test': test_df}
for dataset, df in dataframes.items():
    csv_path = os.path.join(output_dir, f"{dataset}_dataframe.csv")
    df.to_csv(csv_path, index=False)
    print(f"{dataset.capitalize()} DataFrame saved as {csv_path}")

Train DataFrame saved as /workspaces/PP5-Cherry-Leaves/outputs/train_dataframe.csv
Validation DataFrame saved as /workspaces/PP5-Cherry-Leaves/outputs/validation_dataframe.csv
Test DataFrame saved as /workspaces/PP5-Cherry-Leaves/outputs/test_dataframe.csv
